# 爬虫的AJAX

# 1. AJAX简介

① 网页在往下面拖的时候，AJAX可以使得动态加载。

② 什么是AJAX: AJAX(Asynchronouse JavaScript And XML) 即异步的JavaSscript和XML。Ajax可以使网页实现异步更新，这意味着可以在不重新加载整个网页的情况下，对网页的某部分进行更新。

③ 对于传统的网页，如果需要更新内容，那么必须要刷新整个网页页面。但有了Ajax，便可以在页面不必全部刷新的情况下更新其内容。在这个过程中，页面实际上是在与后台的服务器进行了数据交互，获取到数据之后，再利用JavaScript改变网页，这样网页内容就好更新了。

注意：进入某用户微博网址→Network→XHR→不一定是第一行的响应。

## 1.1 爬取新闻文本和图片数据

In [ ]:
#coding=utf-8
import requests
import os
import csv

'''
title,type,date,image
'''

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"}   # 这个header要重写
    
def write_scv(filename,dictdata):
    with open(filename,'a',encoding='utf-8-sig',newline='') as csvf:
        fieldnames = ['title','summery','channelStr','intervalStr','image']
        writer = csv.DictWriter(csvf,fieldnames=fieldnames)
        writer.writerow(dictdata)
        
def creat_dir(dir_name):   # 创建一个图片的文件夹
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

# 保存图片方法
def save_image(dir_name,url):
    image_res = requests.get(url,headers)  # url是图片的连接，requests.get()为请求获得图片信息      
    image_name = os.path.basename(url)    # 图片的名字
    image_path = os.path.join(dir_name,image_name)  # 文件夹的路径和图片名字拼接在一起                           # 文件夹名字和文件名字拼接在一起 
    with open(image_path,'wb') as f:  # image_name可以用绝对路径，可以用相对路径，wb表示写入二进制        
        f.write(image_res.content)    # 二进制的用content，图片和视频都用content
        
if __name__=='__main__':
    dir_name = 'image'
    creat_dir(dir_name)    # 创建文件夹名为dir_name的文件夹
    start_url='https://www.ofweek.com/ai/CATList-201700-8100-ai-{}.html'
    urls = [start_url.format(i) for i in range(1,523)]   # 有500多页
    for url in urls:
        res = requests.get(url,headers=headers)
        news = res.json()['newsList']  # res.json()为一页新闻数据，以一个字典形式                     
        #print(news)
        #print(type(res.json())) # 是一个字典
        #print(type(res.json()['newsList'])) # 获得字典键'newsList'对应的值，对应的值是一个列表         
        #print(news[0])   # news[0]为该页第一篇新闻预览数据
        for new in news:
            new_dict = {}
            new_dict['title'] = new['title']             # 标题
            new_dict['summery'] = new['summery']         # 摘要
            new_dict['channelStr'] = new['channelStr']   # 类别 (产业新闻、编辑视点.....)
            new_dict['intervalStr'] = new['intervalStr'] # 发布时间
            
            if new.get('listpic1'):   # 如果有图片，把图片保存到文件夹下面
                image_url1 = new.get('listpic1')
                save_image(dir_name,image_url1)
            if new.get('listpic2'):   # 如果有图片，把图片保存到文件夹下面
                image_url1 = new.get('listpic2')
                save_image(dir_name,image_url1)
            if new.get('listpic3'):   # 如果有图片，把图片保存到文件夹下面
                image_url1 = new.get('listpic3')
                save_image(dir_name,image_url1)
            
            new_dict['image'] = new.get('listpic1')+new.get('listpic2')+new.get('listpic3')
            write_scv('news.csv',new_dict)   # 保存文本信息

## 1.2 打印豆瓣爬取文本数据

In [1]:
# 爬取豆瓣数据
#coding=utf-8
import requests

'''
id,title,actors,score,types,rank,release_date,cover_url
'''

# 当 AJAX 异步加载时，更新页面后，比较 url，可以得出不同的页面的 url 的规律
# 有的页面中的 url 是几个拼接出来，要找一下拼接规律
# 有的页面中的 url 最后一部分有签名 signature,它们是由访问时间戳、文章 ID、用户名密码....组合生成，每个网页都不一样，所以把后面不同的部分删除掉
# 如果删除掉能爬就爬，不能爬就算了，有的能爬但是只能爬一部分，有些字段部分加密了

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}         
start_url = 'https://movie.douban.com/j/chart/top_list?type=24&interval_id=100%3A90&action=&start={}&limit=20'     

page = 0

print("1111")
while True:
    url = start_url.format(page)
    res = requests.get(url,headers=headers)
    print(res.status_code)
    movies = res.json()    # 存的是所有电影的数据
    if not movies:         # 如果爬到的数据为空的，就退出循环
        print(page)
        break
    for movie in movies:
        print(movie['id'])
        print(movie['title'])
        print(movie['actors'])
        print(movie['score'])
        print(movie['types'])
        print(movie['release_date'])
        print(movie['cover_url'])
    page += 20
    break

1111
200
5133063
憨豆先生精选辑
['罗温·艾金森', '保罗·布朗', '理查德·布赖尔斯', '安格斯·迪顿', '罗宾·德里斯科尔', '卡罗琳·昆汀', '鲁道夫·沃克尔', '理查德·威尔逊']
9.6
['喜剧']
1997
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2354179225.jpg
1292063
美丽人生
['罗伯托·贝尼尼', '尼可莱塔·布拉斯基', '乔治·坎塔里尼', '朱斯蒂诺·杜拉诺', '赛尔乔·比尼·布斯特里克', '玛丽萨·帕雷德斯', '霍斯特·布赫霍尔茨', '利迪娅·阿方西', '朱利亚娜·洛约迪切', '亚美利哥·丰塔尼', '彼得·德·席尔瓦', '弗朗西斯·古佐', '拉法埃拉·莱博罗尼', '克劳迪奥·阿方西', '吉尔·巴罗尼', '马西莫·比安奇', '恩尼奥·孔萨尔维', '吉安卡尔洛·科森蒂诺', '阿伦·克雷格', '汉尼斯·赫尔曼', '弗兰科·梅斯科利尼', '安东尼奥·普雷斯特', '吉娜·诺维勒', '理查德·塞梅尔', '安德烈提多娜', '迪尔克·范登贝格', '奥梅罗·安东努蒂', '沈晓谦', '张欣']
9.5
['剧情', '喜剧', '爱情', '战争']
2020-01-03
https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2578474613.jpg
1303408
福尔摩斯二世
['巴斯特·基顿', '凯瑟琳·麦奎尔', '乔·基顿', 'Ward Crane', 'Jane Connelly', 'George Davis', 'Doris Deane', 'Betsy Ann Hisle', '丘比·摩根', 'John Patrick', 'Ford West']
9.5
['喜剧', '动作', '爱情']
1924-04-21
https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2206737207.jpg
30203557
黄子华栋笃笑之金盆𠺘口
['黄子华']
9.5
['喜剧']
2018-07-06
https://img

## 1.3 豆瓣文本数据保存数据库

### 1.3.1 PythonMysql类

In [2]:
# 数据库类
import pymysql
class PythonMysql(object):
    # 构造方法
    def __init__(self,host,user,password,port):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.mysql_db = self.connect_mysql()
    
    # 连接数据库
    def connect_mysql(self):
        try:
            mysql_db = pymysql.connect(host=self.host,
                                       user=self.user,
                                       password=self.password,
                                       port=self.port)
            print('数据库连接成功')
            return mysql_db
        except Exception as e:
            print(e)
            print('数据库连接失败')
            return None
        
    # 创建表
    def create_table(self,create_table_sql):
        try:
            cursor =  self.mysql_db.cursor()
            cursor.execute(create_table_sql)
            print('创建表成功')
        except Exception as e:
            print(e)
            print('创建表失败')
            
    # 查询数据
    def select_mysql(self,qurey_sql):
        cursor =  self.mysql_db.cursor()
        cursor.execute(qurey_sql)
        return self.mysql_db.cursor.fetchall()
    
    # 插入数据
    def insert_mysql(self,insert_sql):
        try:
            cursor =  self.mysql_db.cursor()
            cursor.execute(insert_sql)  # 执行插入语句
            self.mysql_db.commit() # commit方法跟在增删改查后面，意味着把语句提交给数据库
        except Exception as e:
            print(e)
            print(insert_sql)
            self.mysql_db.rollback()

### 1.3.2 Config类

In [3]:
# 配置类
class Config(object):
    host = '127.0.0.1' #localhost
    user = 'root' 
    password = 'password'
    port = 3306
    image_dir = 'iamge1'

### 1.3.3 主文件

In [5]:
#coding=utf-8
import pymysql
import requests    
    
'''
id,title,actors,score,types,rank,release_date,cover_url
'''

config_obect = Config()  # 由于可能数据在多个数据中，所以没有写死
mysqlobject = PythonMysql(config_obect.host, 
                          config_obect.user, 
                          config_obect.password, 
                          config_obect.port)

# 一般会先把语句在数据库中先运行一下，没有报错再移植进Python中
# comment XXXX 为对前面的字段(title)的注释
create_table_sql = '''
                   create table if not exists ai.movies
                   (
                   id int comment '电影id',
                   title varchar(500) comment '电影名',   
                   actors varchar(2000) comment '导演',
                   score varchar(15) comment '评分',
                   types varchar(50) comment '电影类型',
                   movie_rank int comment '电影豆瓣排名',
                   release_date varchar(50) comment '上映时间',
                   cover_url varchar(100) comment '电影海报'
                   )
                  '''

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"}
start_url = 'https://movie.douban.com/j/chart/top_list?type=24&interval_id=100%3A90&action=&start={}&limit=20'     
page = 0 

mysqlobject.create_table(create_table_sql)  # 创建表


while True:
    url = start_url.format(page)
    res = requests.get(url,headers=headers)
    movies = res.json()
    if not movies:
        print(page)
        break
    for movie in movies:
        insert_sql = '''
                     insert into ai.movies values({},'{}',"{}",'{}','{}',{},'{}','{}')
                     '''                                 
        insert_sql = insert_sql.format(movie['id'],
                                       movie['title'],
                                       ','.join([i.replace(r'"','') for i in movie['actors']]), # 名字里面的双引号变为空格，然后再以逗号把字符串连接起来                
                                       movie['score'],
                                       ','.join(movie['types']),
                                       movie['rank'],
                                       movie['release_date'],
                                       movie['cover_url'])
        mysqlobject.insert_mysql(insert_sql)
    page += 20   

数据库连接成功
创建表成功
560


## 1.4 爬取药物通数据

In [6]:
#coding=utf-8
import requests 
from lxml import etree 
start_url = 'http://ypk.39.net/jiating/p{}/'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"}
urls = [start_url.format(i) for i in range(1,12)]  # "家庭常备" 所有页码100    

def spider_drug_detail(url):
    res = requests.get(url,headers=headers)
    html = etree.HTML(res.text)
    print('adopt:',html.xpath('//*[@id="nav-all"]/div[1]/div[1]/div[2]/ul/li[1]/div/p/text()'))     
    print('num:',html.xpath('//*[@id="nav-all"]/div[1]/div[1]/div[2]/ul/li[4]/div/text()'))
    comments = html.xpath('//*[@id="commentshow"]/ul/li') # 获得所有评价
    for comment in comments:
        comment = comment.xpath('./p/text()')
        if comment:
            print('comment:',comment)
    pass

for url in urls:
    res = requests.get(url) 
    html = etree.HTML(res.text)
    drugs = html.xpath('//*[@id="nav-all"]/div[2]/div[2]/div[2]/div/ul/li')    # 一页药物的 xpath 节点                        
    for drug in drugs:
        drug_image = drug.xpath('./a/img/@src')
        drug_name =drug.xpath('./a/img/@alt')
        drug_function = drug.xpath('./div/p[2]/text()')
        drug_price = drug.xpath('./div/p[4]/span/text()')
        drug_detail_url = drug.xpath('./div/p[1]/a/@href')  # 每一个药物名字的超链接                                        
        if drug_detail_url:
            spider_drug_detail(drug_detail_url[0])

adopt: ['疼痛  落枕  骨质增生  肩周炎  腰肌劳损']
num: ['外用。将小袋内润湿剂均匀涂于药芯表面，润湿后直接敷于患处或穴位。每贴敷24小时。']
comment: ['因有肩周炎，膝盖处蹲起经常疼痛，后用奇正消痛膏三贴在患处，不料十天后，贴药处皮肤过敏，症状已持续十几天，奇痒难寐，请专家指点。谢谢！']
comment: ['我在不同的地方买的奇正贴膏包装有差异，一个条码是6933692557191，另一个是6933692513463，我不知怎样辩真假，请告诉我谢']
comment: ['对急性的伤痛效果非常好']
comment: ['在用，还不知道效果怎样。']
comment: ['孩子身上长了好多红疙瘩，尤其是胸口处，看起来还有点化脓，我就赶紧带她去了医院，检查了之后，医生说孩子长的是红斑狼疮，于是就开了这个药，吃了有一周时间，孩子痘...']
comment: ['患有风湿性关节炎，关节经常疼，下雨天更甚，去药店买这药，用了后感觉效果还好。']
adopt: ['冷痛  乏力  屈伸不利  肿胀  痹证']
num: ['口服，一次4粒，一日3次。']
comment: ['效果特别好，特别是风湿性关节炎']
comment: ['这药治月子受凉引起的肩膀 关节疼么？']
comment: ['风湿病多年，关节有点变形，行动不方便，吃西药消炎止疼，外贴膏药，有一点效果，药一停就不行，购买双蚁祛湿通络胶囊9盒，他们有膏药赠送，显效比较慢，但吃药3天感觉明显，关节已经可以活动，晨僵减轻，感觉效果不错。']
comment: ['药效还是可以的，就是副作用有点大，不太敢长期使用!']
comment: ['药物的疗效很好，但是有没有副作用暂时看不出来']
comment: ['双仪祛湿通络将囊的价格是多少？']
adopt: ['口干  鼻塞  疼痛  咳嗽  风热感冒  感冒']
num: ['用开水冲服。一次14克，一日3次；二天为一疗程。']
comment: ['还可以，没有副作用，价格也不是很贵，主要把病治好了就行。']
comment: ['这几天天气变冷，一不注意就感冒了，去药店买感冒药，因为还要上班，吃些西药会让人晕晕欲睡，就跟店员说有没有感冒药吃了后不影响工作的，店员说那就吃中成药的感冒药，推荐了双蚁牌复方感冒灵颗粒给

adopt: ['类风湿关节炎，狼疮性肾炎']
num: ['⑴成人类风湿性关节炎：口服。由于来氟米特半衰期较长，建议间隔24小时给药。为了快速...']
comment: ['服后引起腹泻乍办？']
comment: ['药效一般价格很贵']
comment: ['太贵了吃不起']
comment: ['几个月后有效果价格55元']
comment: ['效果还没有感觉，还需要持续的关注其他方面的效果']
comment: ['药效不明显，价格贵，越来越吃不起了，怎么没有加入医保，而且吃了掉头发，夫妻性冷淡']
adopt: ['头晕  颈椎病']
num: ['开水冲服。一次1-2袋。一日2次，饭后服。']
comment: ['我颈椎间盘突出，服用颈复康颗粒加用VB1和VB12几天手臂麻木减轻，不知道服用多长时间？哪里医院看颈椎病好？']
comment: ['由于颈椎引起的眩晕和肩背痛，吃了5盒颈复康配合西比林，效果不错，不知还要继续服药多长时间？']
comment: ['我妈妈今天49岁，颈椎一直不好，后来同事吃了颈椎康颗粒说挺管用的，我就给我妈妈买了3个疗程的药，最近刚吃几天，但发现手一凉着一冻着就红，然后很痒，我想问问跟吃这个药有关吗？还应不应该吃呢？']
comment: ['我是颈椎问题，吃了20多天了，现在明显好转了，效果还是可以的。']
comment: ['跟本就是假的我吃了几盒没有一点效果，真的药吃几天就会有效果的，还说要分疗程，朋友们注意了不管什么药分疗程吃的切对是假药']
comment: ['别听广告了，无效果，找正规医院治疗。']
adopt: ['恶寒  高热  鼻塞  咳嗽  感冒  流行性感冒']
num: ['口服。一次6～8片，一日3～4次。']
comment: ['这药治疗感冒还可以，效果挺不错的，价格不贵，可以接受。']
comment: ['非常有效，感冒初起时，吃它，症状马上就缓解，不那么难受了。']
comment: ['治感冒很好']
adopt: ['瘀血  关节疼痛  疼痛  刺痛  瘀斑  痹证']
num: ['口服。一次5片，一日3次，或遵医嘱。']
comment: ['资料不全啊']
comment: ['不知道什么地方卖']
comment: ['在沈阳，怎么买不到，去哪能买到']
comm

adopt: ['肿胀  疼痛']
num: ['口服,一次1～2粒,一日3次.']
comment: ['这个药怎么多钱的都有啊，有8元多的，还有30多的，我买的28，到底哪个是真的啊。']
comment: ['请问：此药含激素吗，常年吃对身体有何副作用?']
comment: ['价格高的与低的有何曲別？']
comment: ['肩周炎又犯了，肩膀疼的我胳膊都抬不起来，而且肩膀也有点肿胀，于是我就买了这个药，回来擦了几天，肩部就感觉没有那么疼了，胳膊也能抬起来了。']
adopt: ['感冒  流行性感冒  偏头痛  痛经  疼痛  头痛  关节痛  牙痛  神经痛']
num: ['口服。6岁以上儿童一次0.5～1片；成人一次1～2片，一日3次。']
comment: ['感冒有点微烧，临急去超市买到这药。吃完睡觉，烧倒是退了，但副作用挺大了，整个人都很憔悴，没力。']
comment: ['这是止痛药还是退烧药？退烧效果一般，止痛效果很好~！']
comment: ['高烧，吊针之前吃了两片，止疼效果比较明显，然后一直吃一片剂量没啥事。']
comment: ['头痛吃了.很好的效果']
comment: ['吃完全身没力气脖子疼  也不知道吃的多了还是']
comment: ['还没开始用，不知道效果怎么样。\n']
adopt: ['感冒  流行性感冒  偏头痛  痛经  疼痛  头痛  牙痛  神经痛  关节痛']
num: ['口服。成人一次1片，若症状不缓解，间隔4-6小时可重复用药一次，24小时内不超过4次。']
comment: ['加合百服宁这药治疗感冒真的不错，效果很好，没有副作用。\n']
comment: ['发烧39度，退烧疗效不错，吃了一粒就没有再烧']
comment: ['为什么只能选感冒？\r\n为什么？？？？？？？？？？？？？？？？？？？？？？？？']
comment: ['治疗中年人头疼效果很好']
comment: ['暂时效果还不是太明显，再观察看看。\n']
comment: ['这个药不好买，一般小药店都没有\n']
adopt: ['眩晕  呕吐  晕动病  恶心']
num: ['口服。一次1片，于乘车前20分钟服用。']
comment: ['很管用疗效很好']
comment: ['吃了基本就不晕车了'

adopt: ['风寒湿痹  瘀血  关节肿痛  瘀斑  湿痹  血痹']
num: ['贴患处。12～24小时更换一次。']
comment: ['经常是做剧烈运动的时候受伤，尤其是踢球的时候，总会使用这个药的，效果很好，就是味大了点，现在年轻人没几个像我这样用这药的了。哈哈']
comment: ['感觉用了很不舒服的感觉']
comment: ['感觉自己湿气很重，时有膝关节疼痛，所以买来试试，感觉1天效果不明显2']
comment: ['感觉还行。但是不知道你们用的怎么样。']
adopt: ['瘀血  疼痛  肿胀']
num: ['外用。喷于患处，每日4～6次。']
comment: ['医生开了这个药，用药一段时间后病症有明显的缓解，身体也逐渐的恢复了健康。']
comment: ['搬东西的时候一不小心扭伤了，疼的厉害，去药店买这药，用了后好多了，没有那么痛了。']
adopt: ['疼痛  骨质增生  风湿性关节炎']
num: ['外用。每十二小时左右更换一次，贴患处。']
comment: ['我在药店买了两袋透骨灵橡胶膏，包装上写着是2片，但是打开袋一看是10片，我不知道这个是不是假的']
comment: ['透骨灵橡胶膏,国药准字号两个Z20033233与Z15021517,这个有什么区别吗']
comment: ['有风湿性关节炎，这药就用了几天，关节炎就改善很多了，这药效果不错。']
adopt: ['风寒湿痹  疼痛  湿痹']
num: ['口服。一次3粒，一日3次；2周为一疗程。']
comment: ['加价太高，赚黑心钱啊祝，注意别上当啊··············']
comment: ['不要买，价格贵，疗郊差']
comment: ['没有什么效果，价格大高，黑心药']
adopt: ['疼痛']
num: ['口服。成人一次0.5g（2片）,一日1次，或遵医嘱。']
comment: ['效果差，不是很满意。']
comment: ['我们这里药店没有，怎样才能买到？']
comment: ['这药治疗风湿关节炎没什么效果，还有呼吸困难、耳鸣、恶心、头晕这些副作用。']
comment: ['沈阳哪个药房有卖的，医院也行']
adopt: ['疼痛  关节肿痛  风湿性关节炎']
num: ['外用。贴于患处.']

adopt: ['扭伤']
num: ['外用。贴于患处。']
comment: ['此药膏不错，效果很明显']
comment: ['这药膏基本上每个药房都有，很好用，我也经常买个我妈用。']
comment: [' 请问这个要怎么分辨真假？\n']
comment: ['这种需要长期调理的疾病，不知道吃了会不会副作用特别大呢?']
comment: [' 药物的疗效很好，但是有没有副作用暂时看不出来\n']
comment: ['不错的药效，真的很不错']
adopt: ['骨关节炎']
num: ['口服。一次240mg-480mg(1-2片)，1日3次，或遵医嘱。根据病人病情，连续服用4-12周；如有必要...']
comment: ['母亲关节炎很久了，一直复发，听说这个药对关节炎疗效不错就买来试试看']
comment: ['我想问一下，有糖尿病人能吃吗？']
comment: ['不吃就腿疼，吃了就管用，']
comment: ['我当天吃了就全身皮肤过敏，胸口，大腿，手，后背长满红斑，后背超严重']
comment: ['不如维骨力，吃了病情加重']
comment: ['商家暴利，疗效一般    别上当']
adopt: ['\r\n', '，', '，', '，地龙，', '。                        ']
num: []
comment: ['这是好药   烧伤烫伤 生肌 褥疮 大面积创伤效果很好']
comment: ['我妹妹2岁时在玩火时不小心被火烧伤了脸部,我妹妹现在34岁了现在还有点轻微伤疤,我妹妹现在感到很自卑不知道哪里有治疗我妹妹的烧伤的药物']
comment: ['今天女儿抓碗，被热饭烫了，不知道效果怎么样']
comment: ['很不错的烫伤药，效果非常好！']
comment: ['我的是铝水烫伤了脚背有鸡蛋那么大一块，用了几天的这个药都没有用，有红有仲有疼路都走不了，都不知道要用什么药才好']
comment: ['挺好的药，效果挺不错']
adopt: ['口干  口苦  溃疡  口腔溃疡']
num: ['喷患处，每次适量，一日数次。']
comment: ['开喉剑喷雾剂儿童最多能用几天']
comment: ['现在我有咽喉肿痛，咳嗽呕吐等症状\r\n']
comment: ['开喉剑是我

adopt: ['关节肿痛  神经痛  麻痹  坐骨神经痛  脑血栓  脑血栓形成  血栓形成']
num: ['口服。一次2片，一日3次；小儿酌减。']
comment: ['跑步拉伤了韧带，右脚感到无比的疼痛，赶紧去药店买了这药贴上去，渐渐右脚的疼痛才减轻了。']
comment: ['最近风湿病又犯了，难受的不行，去药店买这药，用了后感觉没有那么痛了。']
comment: ['家里老人有风湿，关节经常会又肿又痛，腿麻，去药店买这药，用了后关节就没有那么痛了，舒服些了。']
adopt: ['晕动病  呕吐  晕车  晕机  恶心']
num: ['口服。成人一次1片。预防晕动病在出发前30分钟服药，治疗晕动病时每4小时服药1次。1日用...']
comment: ['一次最大剂量']
comment: ['对神经不好']
comment: ['不错，有点效果，坚持']
adopt: ['疼痛  活动受限  肌肉萎缩  瘀斑  缺血性坏死']
num: ['口服。每次10克，一日3次，3个月为一疗程。']
comment: ['我用了两年，没有明显好转，不好不坏。']
comment: ['谢谢它也谢谢那位好心人，它让我的脚灵活了很多，脚也好了很多，用了一年多没在用了，不错，过了五年我想要买，因为脚还会痛，现在我想把治好，可不知哪里在买']
comment: ['疗效不太好，价格有点贵，希望贵厂在质量上在严格把关']
adopt: ['骨关节炎']
num: ['口服。如果医师处方中没有特殊要求，建议每次2粒胶囊，每日3次(早晨及进餐时)；连续用...']
adopt: ['肩周炎  颈椎病  脊髓肿瘤  硬化症  脊髓小脑变性症  变性  神经症  紧张  痉挛  麻痹  硬化  轻瘫']
num: ['饭后口服。通常成人一次1片，一日3次。或遵医嘱。']
comment: ['囖就行就行了呢还是不知道除了这句']
comment: ['我是一个痉挛性脊髓麻痹而导致痉挛性麻痹的患者，自从每日准时吃三次盐酸乙哌立松片，血液循环得到了充分改善，麻痹少了，痉挛性抽经频率也下降了，大家都觉得我正常了很多。']
comment: ['有血压下降']
adopt: ['疼痛  扭伤']
num: ['成人及2岁以上儿童外用。均匀涂抹于疼痛部位，每次l-2个黄豆粒大小的用

## 1.5 爬取头条数据

In [7]:
#coding=utf-8
import requests 
import os 
from lxml import etree
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"}
start_url = 'https://www.toutiao.com/api/search/content/?aid=24&app_name=web_search&offset=0&format=json&keyword=%E8%A1%97%E6%8B%8D&autoload=true&count=20&en_qc=1&cur_tab=1&from=search_tab&pd=synthesis&timestamp=1617949065442&_signature=_02B4Z6wo00f012XXXXAAAIDDuGcFHroCV8dl81nAALklVDAqrM9uF5UIXl9GoN-f144jDUaO9DyuM6DXLdH2yzmVShA7-0VnzM9WTbkoHrY-TvjGcGwS1oJ.0cKTFmJnQFtSK3R8x-hJAQvSde'                    
page = 0 

def creat_dir(dir_name):   # 创建一个图片的文件夹
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

def save_text(pathf,url):
    res = requests.get(url,headers = headers)
    #print(res.text)
    text = etree.HTML(res.text)
    print(text.xpath('//*[@id="root"]/div/div[2]/div[1]/div[2]/article/p/text()'))               
    
dir_name = 'new_data'
creat_dir(dir_name)

while True:
    url = start_url.format(page) 
    res = requests.get(url,headers=headers) 
    datas = res.json()["data"] 
    for data in datas:
        if data.get("title"):
            file_name = '{}.txt'.format(data.get("item_id")) 
            text_url = data.get("ttsearch_msite_url")
            file_path = os.path.join(dir_name,file_name)
            title = '{}.txt'.format(data.get("title")) 
            print(title)
            save_text(file_path,text_url)
        pass
    break

街拍：这是我见到最高的颜值.txt
[]
【街拍北京】暴走17公里，我终于知道街拍为什么这么难了.txt
[]
超好看的国内街拍，每一套都很值得借鉴，不输韩国潮人.txt
[]
街拍：一些小细节也许能够成为关键，可以彰显出不同的魅力.txt
[]
街拍：潮人们的穿搭总是非常好看，她们的生活也是多姿多彩.txt
[]
图虫人文摄影：街拍.txt
[]
一周明星街拍大盘点，美不美主要看气质.txt
[]
街拍美女。.txt
[]
街拍：有气质的女人，她们整体总是能够给人一种很好的感觉.txt
[]
街拍：把“丝巾”打个结穿身上，小姐姐还配一头海浪卷，太迷人了.txt
[]
街拍：这牛仔裤穿得有点…….txt
[]
街拍：尽情地释放自己的想象力，设计出属于自己的穿搭造型.txt
[]
同样是街拍，为什么别人能拍出“电影感”？.txt
[]
街拍：效果不输高跟的平底鞋美女.txt
[]
街拍：拉近自己与时尚之间的距离，多了解一些当下的时尚元素.txt
[]
街拍：生活需要调节才能够保持平衡，时尚能够给你的生活增添趣味.txt
[]
街拍：有的女人总是能够给别人传递出一种清新优雅的感觉.txt
[]


请输入你想下载的内容：水冰月
请输入你想爬取的页数：10
1.jpg 下载成功啦！！！
2.jpg 下载成功啦！！！
3.jpg 下载成功啦！！！
4.jpg 下载成功啦！！！
5.jpg 下载成功啦！！！
6.jpg 下载成功啦！！！
7.jpg 下载成功啦！！！
8.jpg 下载成功啦！！！
9.jpg 下载成功啦！！！
10.jpg 下载成功啦！！！
11.jpg 下载成功啦！！！
12.jpg 下载成功啦！！！
13.jpg 下载成功啦！！！
14.jpg 下载成功啦！！！
15.jpg 下载成功啦！！！
16.jpg 下载成功啦！！！
17.jpg 下载成功啦！！！
18.jpg 下载成功啦！！！
19.jpg 下载成功啦！！！
20.jpg 下载成功啦！！！
21.jpg 下载成功啦！！！
22.jpg 下载成功啦！！！
23.jpg 下载成功啦！！！
24.jpg 下载成功啦！！！
25.jpg 下载成功啦！！！
26.jpg 下载成功啦！！！
27.jpg 下载成功啦！！！
28.jpg 下载成功啦！！！
29.jpg 下载成功啦！！！
30.jpg 下载成功啦！！！
31.jpg 下载成功啦！！！
32.jpg 下载成功啦！！！
33.jpg 下载成功啦！！！
34.jpg 下载成功啦！！！
35.jpg 下载成功啦！！！
36.jpg 下载成功啦！！！
37.jpg 下载成功啦！！！
38.jpg 下载成功啦！！！
39.jpg 下载成功啦！！！
40.jpg 下载成功啦！！！
41.jpg 下载成功啦！！！
42.jpg 下载成功啦！！！
43.jpg 下载成功啦！！！
44.jpg 下载成功啦！！！
45.jpg 下载成功啦！！！
46.jpg 下载成功啦！！！
47.jpg 下载成功啦！！！
48.jpg 下载成功啦！！！
49.jpg 下载成功啦！！！
50.jpg 下载成功啦！！！
51.jpg 下载成功啦！！！
52.jpg 下载成功啦！！！
53.jpg 下载成功啦！！！
54.jpg 下载成功啦！！！
55.jpg 下载成功啦！！！
56.jpg 下载成功啦！！！
57.jpg 下载成功啦！！！
58.jpg 下载成功啦！！！
59.jpg 下载成功啦！！！
60.jpg 下载成功啦！！！
61.jpg 下载成功啦！！！
62.j